# Email DEMO

In [8]:
import pandas as pd
import google.generativeai as genai
import os
import smtplib
from email.message import EmailMessage
from dotenv import load_dotenv

In [9]:
demo_data = pd.DataFrame([
    {
        "name": "Ashelyn",
        "recipient_email": "ashelynkyne@gmail.com",
        "loyalty_profile": "Freedom Seeker",
        "tenure": 2,
        "primary_driver": "cat__Contract_Month-to-month"
    },
    {
        "name": "Michael (The At-Risk VIP)",
        "recipient_email": "estebangr2711@gmail.com",
        "loyalty_profile": "At-Risk VIP",
        "tenure": 24,
        "primary_driver": "num__MonthlyCharges"
    },
    {
        "name": "Jessica",
        "recipient_email": "estebangr2711@gmail.com",
        "loyalty_profile": "New & Uncertain",
        "tenure": 6,
        "primary_driver": "num__tenure"
    }
])

demo_data

,name,recipient_email,loyalty_profile,tenure,primary_driver
0,Ashelyn,ashelynkyne@gmail.com,Freedom Seeker,2,cat__Contract_Month-to-month
1,Michael (The At-Risk VIP),estebangr2711@gmail.com,At-Risk VIP,24,num__MonthlyCharges
2,Jessica,estebangr2711@gmail.com,New & Uncertain,6,num__tenure


In [10]:
# === 1. CONFIGURATION AND SETUP ===

# --- Load Credentials Securely from .env file ---
load_dotenv()

try:
    SENDER_EMAIL = os.environ["EMAIL_SENDER"]
    SENDER_PASSWORD = os.environ["EMAIL_PASSWORD"]
    genai.configure(api_key=os.environ["GEMINI_API_KEY"])
except KeyError as e:
    print(f"FATAL ERROR: Environment variable {e} not found.")
    print("Please ensure you have a .env file with EMAIL_SENDER, EMAIL_PASSWORD, and GEMINI_API_KEY.")
    exit()

In [11]:
MASTER_PROMPT = """
You are an expert marketing copywriter for Vodafone, a top-tier telecommunications company.
Your task is to write a personalized, persuasive, and friendly email to a valued customer who our data suggests might be at risk of leaving. The goal is to proactively address their potential concerns and retain them with a relevant offer.

### STRICT GUIDELINES TO FOLLOW ###
1. Tone: Friendly, Clear, Positive, Professional, Trustworthy.
2. Structure:
   - Start with "Subject: [Your Subject Line Here]" on the very first line.
   - Follow with a blank line.
   - Then, begin the body of the email with "Hi {customer_name},"
   - End with the signature "Best regards,\nThe Vodafone Customer Care Team"
3. Constraints:
   - DO NOT mention "churn," "risk," "leaving," or "canceling."
   - The tone must be 100% positive and proactive.

### CUSTOMER DATA & TASK ###
- Customer Name: {customer_name}
- Customer Since: {tenure_months} months
- Primary Churn Driver (For your context only, do not mention directly): {primary_churn_driver}
- Suggested Offer to Weave into the Email: {suggested_offer}

Now, write the complete email.
"""

In [12]:
# === 3. CORE FUNCTIONS (Unchanged) ===
def get_prompt_inputs(customer_row):
    profile = customer_row['loyalty_profile']
    suggested_offer = ""
    if profile == "Freedom Seeker":
        suggested_offer = "A special loyalty discount on their monthly bill if they switch to a 12-month plan, offering both savings and price stability."
    elif profile == "At-Risk VIP":
        suggested_offer = "A complimentary plan review with an expert to ensure they're on the best-value tier, plus a free month of our Premium Tech Support service as a thank you."
    elif profile == "New & Uncertain":
        suggested_offer = "A welcome gift of a free 3-month trial of our Premium Online Security package to help them get the most out of their services."
    
    # Use the name field for personalization
    customer_name_for_email = customer_row['name'].split(' ')[0]
    
    return {
        "customer_name": customer_name_for_email, "tenure_months": customer_row['tenure'],
        "primary_churn_driver": customer_row['primary_driver'], "suggested_offer": suggested_offer
    }

def generate_email_content(customer_data):
    try:
        model = genai.GenerativeModel('gemini-1.5-pro-latest')
        prompt_inputs = get_prompt_inputs(customer_data)
        final_prompt = MASTER_PROMPT.format(**prompt_inputs)
        response = model.generate_content(final_prompt)
        return response.text
    except Exception as e:
        print(f"Error generating email content: {e}")
        return None

def parse_email_content(full_content):
    try:
        parts = full_content.split('\n\n', 1)
        subject_line = parts[0].replace("Subject: ", "").strip()
        body = parts[1].strip()
        return subject_line, body
    except Exception:
        return "A Special Offer from Vodafone", full_content

def send_email(recipient_email, subject, body):
    print(f"  > Preparing to send email to {recipient_email}...")
    msg = EmailMessage()
    msg['Subject'] = subject
    msg['From'] = SENDER_EMAIL
    msg['To'] = recipient_email
    msg.set_content(body)
    try:
        with smtplib.SMTP_SSL('smtp.gmail.com', 465) as smtp:
            smtp.login(SENDER_EMAIL, SENDER_PASSWORD)
            smtp.send_message(msg)
        print(f" Email sent successfully to {recipient_email}!")
        return True
    except Exception as e:
        print(f" Failed to send email to {recipient_email}. Error: {e}")
        return False

# === 4. MAIN DEMO SCRIPT (Updated to use the DataFrame directly) ===
if __name__ == "__main__":
    print("="*60)
    print("      DEPT AI Customer Retention - LIVE DEMO")
    print("="*60)

    # Safety check for placeholder emails
    if demo_data['recipient_email'].str.contains("recruiter|example.com").any():
        print("WARNING: The demo_data DataFrame contains placeholder emails.")
        print("Please edit the 'recipient_email' column before the live demo.")
        should_send_emails = False
    else:
        should_send_emails = True
        print("Recipient emails are configured. Live sending is enabled.")
        
    print("-" * 60)

    for index, customer in demo_data.iterrows():
        print(f"\n[STEP 1] Identifying customer persona '{customer['name']}'...")
        
        print("[STEP 2] Generating personalized email with Gemini AI...")
        email_content = generate_email_content(customer)
        
        if email_content:
            subject, body = parse_email_content(email_content)
            
            print("--- EMAIL GENERATED ---")
            #print(f"Subject: {subject}")
            #print("---")
            #print(body)
            #print("-----------------------\n")
            
            if should_send_emails:
                recipient = customer['recipient_email']
                print(f"[STEP 3] Sending live email to {recipient}...")
                send_email(recipient, f"DEPT AI Demo: {subject}", body)
            else:
                print("[STEP 3] SKIPPED: Live email sending is disabled due to placeholder recipients.")

            print("-" * 60)
        else:
            print("Skipping email generation due to an API error.")
            print("-" * 60)
            
    print("\n✅ Live Demo Complete.")
    if should_send_emails:
        print("The recruiters should have received the emails.")
    print("="*60)

      DEPT AI Customer Retention - LIVE DEMO
Recipient emails are configured. Live sending is enabled.
------------------------------------------------------------

[STEP 1] Identifying customer persona 'Ashelyn'...
[STEP 2] Generating personalized email with Gemini AI...
--- EMAIL GENERATED ---
[STEP 3] Sending live email to ashelynkyne@gmail.com...
  > Preparing to send email to ashelynkyne@gmail.com...
 Email sent successfully to ashelynkyne@gmail.com!
------------------------------------------------------------

[STEP 1] Identifying customer persona 'Michael (The At-Risk VIP)'...
[STEP 2] Generating personalized email with Gemini AI...
--- EMAIL GENERATED ---
[STEP 3] Sending live email to estebangr2711@gmail.com...
  > Preparing to send email to estebangr2711@gmail.com...
 Email sent successfully to estebangr2711@gmail.com!
------------------------------------------------------------

[STEP 1] Identifying customer persona 'Jessica'...
[STEP 2] Generating personalized email with Ge